In [ ]:
!pip3 install azure-cli
!pip3 install azure-ai-formrecognizer
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential
import os
import pandas as pd

In [21]:
def get_Azure_credential():
    try:
        azureKeyCredential = os.environ.get('Azure_Key_Credential')
    except Exception as exception:
        raise exception
    return azureKeyCredential


def get_recognized_content(endpoint, azureKeyCredential, document, extractedTextDF):
    try:
        credential = AzureKeyCredential(azureKeyCredential)  #"3fd6e3c5269c48959b6074e922a31480"
        client = FormRecognizerClient(endpoint, credential)
        #for document in docList:
        wrdCnt=0
        with open(document, 'rb') as doc:
            contentRecognizer = client.begin_recognize_content(doc)
            response = contentRecognizer.result()
            for content in response:
                for line in content.lines:
                    for word in line.words:
                        wrdCnt+=1
                        textDF = pd.DataFrame([(word.text, word.confidence)], columns = 'Text Confidence_score'.split())
                        extractedTextDF = extractedTextDF.append(textDF)    
                if (content.tables):
                    for table in content.tables:
                        for cell in table.cells:
                            textDF = pd.DataFrame([(cell.text, cell.confidence)], columns = 'Text Confidence_score'.split())
                            extractedTextDF = extractedTextDF.append(textDF)      
            print("Total number of words extracted for document '{}' is {}".format(document, str(wrdCnt)))                                                 
            print('****************************************')      
    except Exception as exception:
        raise exception
    return extractedTextDF               

In [25]:
endpoint = "https://mcriformrecognizer.cognitiveservices.azure.com/"
docList = ["SamplePresc.pdf"]   # List of all documents needed to be analysed by Azure
result= []
extractedTextDF = pd.DataFrame()   

azureKeyCredential = get_Azure_credential()
for document in docList:
    extractedTextDF = get_recognized_content(endpoint, azureKeyCredential, document, extractedTextDF) 
extractedTextDF.to_excel(r'Azureoutput.xlsx',sheet_name='Azure FormRecognizer', index = False)    

Total number of words extracted for document 'SamplePresc.pdf' is 1363
****************************************


In [26]:
extractedTextDF.head()

,Text,Confidence_score
0,1.,0.717
0,WRITE,0.762
0,PATIENT,0.927
0,DETAILS,0.937
0,OR,0.941
